# Config

In [2]:
import requests
import urllib
from bs4 import BeautifulSoup
import bs4
import re
import scholarly
import pandas as pd
import multiprocessing as mp
from joblib import Parallel, delayed

# Scraping Faculty Names

In [45]:
eco_faculy=pd.read_excel('Data/Faculty_names.xlsx',index_col=0)

In [ ]:
## Scraping the faculty name did not work out because of the wierd html tags of facultis in the cology directory
url='https://www.ecology.uga.edu/directory/#faculty'
Page=requests.get(url)
soup=BeautifulSoup(Page.text,'html.parser')
eco_faculty_soup=soup.find_all('div',{'class':'portal-item bio-wrap mix courtesy-faculty g '})
faculty_names_tag=eco_faculty_soup.find_all('span',{'class':'title'})

n_faculty=len(faculty_names_tag)

faculty_names=[]

for i in range(n_faculty):
    
    academic_title=faculty_names_tag[i].parent.parent.p.text
    full_name=faculty_names_tag[i].parent.parent.h5.text
    is_eco=False
    if 'ecology' in faculty_names_tag[0].parent.parent.a['href']:
        is_eco=True
    
    if ('Professor' in academic_title) and (is_eco==True):
        faculty_names.append(full_name)

faculty_names=list(set(faculty_names))

In [ ]:
eco_faculy['GS_link']=np.nan
eco_faculy['academic_title']=np.nan
eco_faculy['GS_keywords']=np.nan
for faculty_name in eco_faculy.index:
    search_query = scholarly.search_author(faculty_name+', uga')
    query_result = next(search_query,'NA')
    if query_result!='NA':
        author = query_result.fill()
        eco_faculy.loc[faculty_name,'GS_link']=author.id
        eco_faculy.loc[faculty_name,'academic_title']=author.affiliation
        eco_faculy.loc[faculty_name,'GS_keywords']=','.join(author.interests)
        print(faculty_name,author.affiliation,'\n\n')

In [65]:
eco_faculy.to_pickle('Data/eco_faculy')
eco_faculy.to_excel('Data/eco_faculy.xlsx')

# Scraping abstract: Faculty with Public Google Scholar Profile

In [3]:
eco_faculty=pd.read_pickle('Data/eco_faculty')
eco_faculty.index=eco_faculty.index.str.replace('.',' ')

In [4]:
def PubsAbs(faculty_name,df_eco_faculty):
    print(faculty_name)
    pub_df=pd.DataFrame(columns=['title','abstract','year'])
    
    search_query = scholarly.search_author(faculty_name+', uga')
    query_result = next(search_query,'NA')
    author = query_result.fill()
    
    for i,pub in enumerate(author.publications):
        print(i)
        pub_meta=pub.fill()
        pub_df.loc[i,'title']=pub_meta.bib['title']
        
        if 'abstract' in pub_meta.bib.keys():
            if type(pub_meta.bib['abstract'])==bs4.element.Tag:
                abs_txt=pub_meta.bib['abstract'].text
            else:
                abs_txt=pub_meta.bib['abstract'] 
            abs_txt=abs_txt.replace('\xa0…','')
            pub_df.loc[i,'abstract']=abs_txt

        if ('year' in pub.bib.keys()):
            pub_df.loc[i,'year']=pub.bib['year']
    pub_df.to_pickle('Faculty pubs/'+faculty_name)
    return 0

In [6]:
n_cores=mp.cpu_count()
pool = mp.Pool(processes=n_cores-2)
remaining_faculty=eco_faculty[eco_faculty.GS_link.notnull()].index

In [7]:
Parallel(n_jobs=n_cores)(delayed(PubsAbs)(faculty_name,eco_faculty) for faculty_name in remaining_faculty)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# Scraping abstract: Faculty with Private Google Scholar Profile

In [40]:
eco_faculty[eco_faculty.GS_link.isnull()].index

Index(['Kimberly Andrews', 'Ford Ballantyne', 'Scott Connelly',
       'Vanessa Ezenwa', 'Laurie Fowler', 'Byron Freeman', 'Benjamin Parrott',
       'O  E  (Gene) Rhodes', 'Jasmine Crumsey Forde'],
      dtype='object')

In [8]:
pd.read_pickle('Faculty pubs/Elizabeth King')

,title,abstract,year
0,Identifying linkages among conceptual models o...,We present an ecological framework for consi...,2006
1,Dryland ecohydrology and climate change: criti...,Drylands cover about 40% of the terrestrial la...,2012
2,A sustainability framework for assessing trade...,"Achieving sustainability, i.e., meeting the n...",2015
3,Egg size is a developmentally plastic trait: e...,This study documents and compares patterns of ...,1997
4,Facilitative Effects of Aloe Shrubs on Grass E...,Under the harsh environmental conditions pre...,2008
5,Trade-offs in ecosystem services and varying s...,In efforts to increase human well-being while...,2015
6,Kenyan pastoralist societies in transition: va...,"In the drylands of northern Kenya, as elsewhe...",2014
7,δ2H isotopic flux partitioning of evapotranspi...,The partitioning of surface vapor flux (FET) ...,2014
8,Facilitative effects of Aloe secundiflora shru...,Livestock overgrazing can threaten the ecologi...,2008
9,Coupling vegetation organization patterns to s...,"[1] In dryland ecosystems, understanding the e...",2011
